In [1]:
import numpy as np
np.set_printoptions(suppress=True)

from SmoothSparseWeights import *
from SmoothSparseWeights_Alternating import *

In [2]:
# exampleName="cheburashka"
# poses_folder="poses-2"
# poses_ind=7
# clip_k=5
# handle_num=11


exampleName="cow"
poses_folder="poses-2"
poses_ind=5
clip_k=7
handle_num=15


# exampleName="wolf"
# poses_folder="poses-1"
# poses_ind=2
# clip_k=18
# handle_num=21


objective_choice=0


weights = {
           'W_sparse': 0.0,
           'W_spatial': 0.0,
           'W_sum': 100000.0
          }

In [3]:
#### read obj file

### rest_poses:
vertices_obj_path="./models/"+exampleName+"-all/"+exampleName+".obj"
mesh0=TriMesh.FromOBJ_FileName(vertices_obj_path)

### other poses
vertices_obj_dir="./models/"+exampleName+"-all/"+poses_folder+"/"
filenames=glob.glob(vertices_obj_dir+"*.obj")
print(filenames)

mesh1_list=[]
for i in range(len(filenames)):
    mesh1_list.append(TriMesh.FromOBJ_FileName(filenames[i]))


vertices1=vertices1_temp=np.hstack((np.asarray(mesh0.vs),np.ones((len(mesh0.vs),1))))
vertices2=vertices2_temp=np.asarray(mesh1_list[0].vs)
for i in range(1,len(mesh1_list)):
    vertices1=np.hstack((vertices1, vertices1_temp))
    vertices2=np.hstack((vertices2, np.asarray(mesh1_list[i].vs)))
print(vertices1.shape) 
print(vertices2.shape) 


['./models/cow-all/poses-2/cow-1.obj', './models/cow-all/poses-2/cow-2.obj', './models/cow-all/poses-2/cow-3.obj', './models/cow-all/poses-2/cow-4.obj', './models/cow-all/poses-2/cow-5.obj', './models/cow-all/poses-2/cow-6.obj', './models/cow-all/poses-2/cow-7.obj']
(11666, 28)
(11666, 21)


In [4]:
#### read our output of simplex_hull.py 
results_path="./results_pipeline/"+exampleName+"-"+str(handle_num)+"/result.txt"
endmembers, mixing_weights=load_result(results_path)
### endmembers shape is handle * poses * 12??

#### this is to make sure the 3*4 or 4*3 error does not happen. ???
# endmembers=endmembers.reshape((-1, 4, 3))
# endmembers=np.transpose(endmembers, (0,2,1))
# endmembers=endmembers.reshape((len(mixing_weights), -1, 12))


endmembers=endmembers.reshape((len(mixing_weights),-1))
mixing_weights=mixing_weights.T


print (endmembers.shape)
print (mixing_weights.shape)
print (mixing_weights[:3])
mixing_weights_clip, grad_zero_indices=clip_first_k_values(mixing_weights, clip_k)
print (mixing_weights_clip[:3])

print (grad_zero_indices[:len(endmembers)*3])

print (mixing_weights_clip.ravel()[grad_zero_indices].sum())

(15, 84)
(11666, 15)
[[0.20570602 0.00042219 0.02655054 0.15167153 0.00018981 0.00004767
  0.00030539 0.00002423 0.02001831 0.00000105 0.00102998 0.14445015
  0.00045419 0.44910447 0.00002447]
 [0.21555833 0.0004373  0.02679411 0.16466939 0.00016431 0.00004939
  0.00029969 0.00002367 0.01424946 0.00000119 0.00101018 0.15187666
  0.00048589 0.42435294 0.00002749]
 [0.24024097 0.00040106 0.02997077 0.15734726 0.00013956 0.00004676
  0.00030475 0.00002278 0.01438586 0.00000122 0.00103343 0.11445163
  0.00051674 0.44110425 0.00003297]]
[[0.20570602 0.         0.02655054 0.15167153 0.         0.
  0.         0.         0.02001831 0.         0.00102998 0.14445015
  0.         0.44910447 0.        ]
 [0.21555833 0.         0.02679411 0.16466939 0.         0.
  0.         0.         0.01424946 0.         0.00101018 0.15187666
  0.         0.42435294 0.        ]
 [0.24024097 0.         0.02997077 0.15734726 0.         0.
  0.         0.         0.01438586 0.         0.00103343 0.11445163
  0.  

In [5]:
#### solve for transformation per vertex, with smoothness regularization and transformation matrix L2 norm regularization.
output_prefix="./sparse-smooth-mixing-weights-test/"+exampleName

In [6]:
scale=find_scale(vertices1)
print (scale)

708.6870049193496


In [7]:
### gt test
val=objective_func_vector(mixing_weights.ravel(), vertices1, vertices2, endmembers, mixing_weights.ravel(), mixing_weights_clip.ravel(), choice=objective_choice)
print (np.sqrt(np.square(val).sum()/(len(val)*scale*scale)))
recovered_vertices2=recover_vertices(mixing_weights, vertices1, endmembers)
print (E_RMS_kavan2010(vertices2, recovered_vertices2, scale))

3.766786638470725e-05
0.07533573276941448


In [8]:
val=objective_func_vector(mixing_weights_clip.ravel(), vertices1, vertices2, endmembers, mixing_weights.ravel(), mixing_weights_clip.ravel(), choice=objective_choice)
print (np.sqrt(np.square(val).sum()/(len(val)*scale*scale)))
recovered_vertices2=recover_vertices(mixing_weights_clip, vertices1, endmembers)
print (E_RMS_kavan2010(vertices2, recovered_vertices2, scale))

8.155936486433758
16311.872972867513


In [9]:
start=time.time()

res, new_endmembers = run(mesh0, mesh1_list, output_prefix, weights, endmembers, mixing_weights.ravel(), mixing_weights_clip.ravel(), grad_zero_indices, initials=mixing_weights_clip.ravel(), choice=objective_choice)

# res, new_endmembers = run(mesh0, mesh1_list, output_prefix, weights, endmembers, mixing_weights.ravel(), mixing_weights_clip.ravel(), grad_zero_indices, initials=(mixing_weights_clip.ravel() + mixing_weights.ravel())/2.0, choice=objective_choice)

res=res.reshape(mixing_weights.shape)
end=time.time()
print("using time: ", end-start)

(11666, 28)
(11666, 21)
####loop:  0
####solve mixing weights
True
4173.3830672145705
####solve endmembers
True
####endmember changes:  [1998.62402881  568.77727215 2188.07340524 1038.27295473    1.30911247
    3.59887847   97.90188146    0.32683428  609.48841298    0.
 3244.250178    585.1506228  2910.43581325 1660.62291916    0.07209785]
####loop:  1
####solve mixing weights
True
207.4278844979567
####solve endmembers
True
####endmember changes:  [ 19.72368518  14.02149064 345.04338341  19.38049273   0.00434099
   0.03607512   4.1472472    0.         147.49194898   0.
  61.1225285   21.19634171  55.78902481  27.960639     0.        ]
####loop:  2
####solve mixing weights
True
141.3587148648052
####solve endmembers
True
####endmember changes:  [ 15.37414723  14.35796449 154.71366502  21.58737978   0.01335737
   0.05488144   4.25499403   0.         118.78742649   0.
  43.24399251  17.50922161  38.40517091  24.80907225   0.        ]
####loop:  3
####solve mixing weights
True
130.2137301

In [10]:
val=objective_func_vector(res.ravel(), vertices1, vertices2, new_endmembers, mixing_weights.ravel(), mixing_weights_clip.ravel(), choice=objective_choice)
print (np.sqrt(np.square(val).sum()/(len(val)*scale*scale)))
recovered_vertices2=recover_vertices(res, vertices1, new_endmembers)
print (E_RMS_kavan2010(vertices2, recovered_vertices2, scale))

0.03027733205742391
60.55466411484781


In [11]:
print (res.reshape((-1, len(endmembers)))[:3]) ### should have same zeros as initial input.
print (res.ravel()[grad_zero_indices].sum()) ### should be zero
print (res.sum())
print ((((1-res.sum(axis=-1))**2).sum()/len(endmembers))*weights["W_sum"]) ##W_sum term

[[0.20547409 0.         0.02717891 0.15141601 0.         0.
  0.         0.         0.02032394 0.         0.00251736 0.14431099
  0.         0.44877751 0.        ]
 [0.2153823  0.         0.02711883 0.16447858 0.         0.
  0.         0.         0.01443057 0.         0.00251752 0.15184779
  0.         0.42422309 0.        ]
 [0.24021859 0.         0.0294954  0.15733729 0.         0.
  0.         0.         0.01461049 0.         0.00251787 0.1146578
  0.         0.44116064 0.        ]]
0.0
11665.992289510848
0.0016879578247123719


In [12]:
print (((1-res.sum(axis=-1))**2).sum()/len(res))
print (res.min(), res.max())

2.17035550923072e-11
0.0 0.9497134763901076


In [13]:
# #### read gt per vertex transformation 
# groundtruth_names=glob.glob("./models/"+exampleName+"-all/"+exampleName+"-*.DMAT")
# print(groundtruth_names)

# gt_all=None
# for i in range(len(groundtruth_names)):
#     gt=load_DMAT(groundtruth_names[i])
#     gt=gt.T
#     gt=gt.reshape((-1,4,3))
#     gt=np.transpose(gt, (0,2,1)).reshape((-1,12))
#     if gt_all is None:
#         gt_all=gt
#     else:
#         gt_all=np.hstack((gt_all, gt))
    
# print(gt_all.shape)


In [14]:
#### read gt per vertex mixing weights 
groundtruth_names=glob.glob("./models/"+exampleName+"-all/"+exampleName+".DMAT")
print(groundtruth_names)
gt=load_DMAT(groundtruth_names[0])
gt_mixing_weights=gt.T.copy()
print (gt_mixing_weights.shape)

print (((gt_mixing_weights.sum(axis=-1)-1)**2).sum())

['./models/cow-all/cow.DMAT']
(11666, 15)
1.5653958587979453e-28


In [15]:
### load groundtruth endmember
groundtruth_names=glob.glob("./models/"+exampleName+"-all/"+poses_folder+"/"+exampleName+"-*.Tmat")
print(groundtruth_names)

gt_handles=None
for i in range(len(groundtruth_names)):
    gt=load_Tmat(groundtruth_names[i])
    gt=gt.reshape((-1,4,3))
    gt=np.transpose(gt, (0,2,1)).reshape((-1,12))
    if gt_handles is None:
        gt_handles=gt
    else:
        gt_handles=np.hstack((gt_handles, gt))
    
print(gt_handles.shape)
gt_handles=gt_handles.reshape(len(groundtruth_names), -1, 12)

# gt_handles=np.transpose(gt_handles.reshape(-1, len(groundtruth_names), 12), (1,0,2)) ### is this true?

print(gt_handles.shape)

for i in range(len(gt_handles)):
    print (gt_handles[i][2].reshape((3,4)))

test=gt_handles[1][2].reshape((3,4))[:,:3]
print (test.T.dot(test))

['./models/cow-all/poses-2/cow-1.Tmat', './models/cow-all/poses-2/cow-2.Tmat', './models/cow-all/poses-2/cow-3.Tmat', './models/cow-all/poses-2/cow-4.Tmat', './models/cow-all/poses-2/cow-5.Tmat', './models/cow-all/poses-2/cow-6.Tmat', './models/cow-all/poses-2/cow-7.Tmat']
(15, 84)
(7, 15, 12)
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[  0.965808   0.145115  -0.21484  -27.4451  ]
 [ -0.210922   0.921663  -0.325653 -18.4378  ]
 [  0.150753   0.359833   0.920757 -33.9079  ]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [16]:
# #### do a groundtruth test.
# val=objective_func_vector(gt_mixing_weights.ravel(), vertices1, vertices2, gt_handles, gt_mixing_weights.ravel(), gt_mixing_weights.ravel(), choice=objective_choice)
# print ((abs(val)).max(), (abs(val)).min())
# print (np.square(val).sum()/(len(val)))

# res=run(mesh0, mesh1_list, output_prefix, weights, gt_handles, gt_mixing_weights.ravel(), gt_mixing_weights.ravel(), grad_zero_indices, initials=gt_mixing_weights.ravel(), choice=objective_choice)
# res=res.reshape(gt_mixing_weights.shape)

In [17]:
### plot handles corresponding to gt mixing weights, recovered mixing weights, and post-processed mixing weights.
###

Vertices=vertices2[:,poses_ind*3:poses_ind*3+3]

gt_handle_position=gt_mixing_weights.T.dot(Vertices)/gt_mixing_weights.T.sum(axis=-1)[...,np.newaxis]
recovered_handle_position=mixing_weights.T.dot(Vertices)/mixing_weights.T.sum(axis=-1)[...,np.newaxis]
cliped_handle_position=mixing_weights_clip.T.dot(Vertices)/mixing_weights_clip.T.sum(axis=-1)[...,np.newaxis]
postprocessed_handle_position=res.T.dot(Vertices)/res.T.sum(axis=-1)[...,np.newaxis]

%matplotlib 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig= plt.figure()
ax=fig.add_subplot(111, projection='3d')

ax.scatter(gt_handle_position[:,0],gt_handle_position[:,1], gt_handle_position[:,2], c='k', s=50)
ax.scatter(recovered_handle_position[:,0],recovered_handle_position[:,1], recovered_handle_position[:,2], c='r', s=50)
# ax.scatter(cliped_handle_position[:,0],cliped_handle_position[:,1], cliped_handle_position[:,2], c='g', s=50)
ax.scatter(postprocessed_handle_position[:,0],postprocessed_handle_position[:,1], postprocessed_handle_position[:,2], c='b', s=50)
ax.scatter(Vertices[:,0],Vertices[:,1], Vertices[:,2], c='y', s=1, edgecolors='none')

plt.show()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


Using matplotlib backend: MacOSX
